In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.sparse as sps
import scipy.io as io

### Read needed files

In [ ]:
# read interactions file
train_final = pd.read_csv('../input/train_final.csv', sep = '\t')
train_final['value'] = 1
print(train_final.head())
print(train_final.shape)